# Ensemble Poincaré Sections

`DoublePendulumSubclassRandomEnsemble` is a class refactor aiming to plot Poincaré sections



In [24]:
from random import sample

import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt

from MathFunctions import *
from DoublePendulumSubclassRandomEnsemble import DoublePendulum, DoublePendulumEnsemble

----
&nbsp;
#### Variable & Parameter Declaration

In [2]:
l1, l2, m1, m2, M1, M2, g = sp.symbols('l1 l2 m1 m2 M1 M2 g', real=True, positive=True)

# Declare functions
theta1 = sp.Function('theta1')(t)
theta2 = sp.Function('theta2')(t)
p_theta_1 = sp.Function('p_theta_1')(t)
p_theta_2 = sp.Function('p_theta_2')(t)

In [3]:
# Parameters
params = {
    g: 9.81,  # Acceleration due to gravity (m/s^2)
    l1: 1.0,  # Length of the first rod (m)
    l2: 1.0,  # Length of the second rod (m)
    m1: 1.0,  # Mass of the first bob (kg)
    m2: 1.0,  # Mass of the second bob (kg)
    M1: 1.0,  # Mass of first uniform rod (kg)
    M2: 1.0   # Mass of second uniform rod (kg)
}

In [4]:
# Time vector

# calculate frames/second
# (time end - time start) * 200 = 24000 steps
# (time end - time start) * 400 = 48000 steps
# (time end - time start) * 800 = 96000 steps

stop = 120
fps = 200  # frames/second
no_steps = stop * fps

time = [0, stop, no_steps]

In [5]:
# Testing simple pendulum instantiation
# Initial conditions (theta1, theta2, p1, p2)

init_values = [0, 120, 0, 0]
pendulum1 = DoublePendulum(parameters=params, initial_conditions=init_values, time_vector=time)

In [7]:
pendulum1.precompute_positions()
#pendulum1.animate_pendulum(appearance='dark')

----
&nbsp;
#### Potential Energy Calculator

We need to find $E_{\text{mech}}$ of the system which is all potential energy when releasing the pendulums from rest

In [6]:
def calculate_potential_energy(theta1_val, theta2_val, parameters, model='simple'):
    """
    Calculate the potential energy of the double pendulum system relative to the datum where theta1 = 0 and theta2 = 0.
    """
    if model == 'simple':
        V = -(m1 + m2) * g * l1 * sp.cos(theta1) - m2 * g * l2 * sp.cos(theta2)

    elif model == 'compound':
        V = -M1 * g * (l1 / 2) * sp.cos(theta1) - M2 * g * ((l1 * sp.cos(theta1)) + (l2 / 2) * sp.cos(theta2))

    else:
        raise ValueError("Model must be 'simple' or 'compound'")

    V = V.subs(parameters)
    V_subst = V.subs({theta1: theta1_val, theta2: theta2_val})
    # Calculate potential energy at theta1 = 0 and theta2 = 0 (datum)
    V_zero = V.subs({theta1: 0, theta2: 0})
    V_relative = V_subst - V_zero

    return V_relative

In [7]:
angle1 = 120
angle2 = 120

theta1_val = np.deg2rad(angle1)
theta2_val = np.deg2rad(angle2)

In [8]:
# Calculate potential energy for the 'simple' model
V_simple = calculate_potential_energy(theta1_val, theta2_val, params, model='simple')
print(f"Potential Energy (Simple Model): {V_simple:.2f} J")

# Calculate potential energy for the 'compound' model
V_compound = calculate_potential_energy(theta1_val, theta2_val, params, model='compound')
print(f"Potential Energy (Compound Model): {V_compound:.2f} J")

Potential Energy (Simple Model): 44.14 J
Potential Energy (Compound Model): 29.43 J


Maximum (theoretical) mechanical energy for this system released from rest is when $\theta_1 = \theta_2 = \pi$

- Potential Energy (Simple Model): 58.86 J
- Potential Energy (Compound Model): 39.24 J

----
&nbsp;
#### `DoublePendulumEnsemble` Instantiation

The below $E_{\text{mech}}$ corresponds to $\theta_1 = \theta_2 = 120^{\circ}$

In [9]:
simple_explorer = DoublePendulumEnsemble(params, time, 'simple', 44.14)
compound_explorer = DoublePendulumEnsemble(params, time, 'compound', 29.42)

DoublePendulumEnsemble initialized with base class.
DoublePendulumEnsemble initialized with base class.


`_run_simulations()` is a protected class method but running for development (will eventually be called by a method that finds the Poincaré points)

In [10]:
simple_explorer._run_simulations()

Special Angle Combinations within energy limit: 39
Final length: 1000
Batch 1 of 10 complete. Time taken: 10.20 seconds.
Batch 2 of 10 complete. Time taken: 9.44 seconds.
Batch 3 of 10 complete. Time taken: 9.21 seconds.
Batch 4 of 10 complete. Time taken: 8.79 seconds.
Batch 5 of 10 complete. Time taken: 8.88 seconds.
Batch 6 of 10 complete. Time taken: 8.77 seconds.
Batch 7 of 10 complete. Time taken: 8.85 seconds.
Batch 8 of 10 complete. Time taken: 8.48 seconds.
Batch 9 of 10 complete. Time taken: 8.48 seconds.
Batch 10 of 10 complete. Time taken: 8.58 seconds.
Simulations Complete. Time taken: 141.50 seconds.


----
&nbsp;
### The Poincaré Section

Check [`solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp) documentation to tweak integrator arguments

In [ ]:
# Define additional parameters for the numerical integration
integrator_args = {
    'rtol': 1e-5,    # default is 1e-3
    'atol': 1e-8     # default is 1e-6
    #'method': 'RK45',
}

With looser tolerances defined above, the solver introduces small errors at each step that accumulate over time, leading to points that are slightly off from their true positions. 

- Tighter tolerances hopefully reduce this drift at the expense of computational load. 

- We want an accurate representation of the system’s dynamics and we want it in a 'reasonable runtime`. These parameters are mutually exclusive

In [ ]:
simple_explorer.find_poincare_section(**integrator_args)

In [ ]:
print(simple_explorer.initial_condition_data.shape)
print(type(simple_explorer.initial_condition_data))
print(len(simple_explorer.initial_condition_data[180]))
print(simple_explorer.initial_condition_data[180][:10])

In [ ]:
print(len(simple_explorer.poincare_section_data[360]))

In [ ]:
simple_explorer.plot_poincare_map()